In [7]:
import chess
import chess.pgn
import glob
import numpy as np

from stockfish import Stockfish
stockfish = Stockfish(path="./stockfish-ubuntu-x86-64-avx2")

path = 'data/5_less.pgn'

num_games = 0
endplay_games = 0

for filepath in glob.glob(path):
    with open(filepath, encoding="utf-8", errors="ignore") as file:
        while(True):
            game = chess.pgn.read_game(file)
            if game is None:
                break
            num_games += 1
            final_board = game.end().board()
            # display(final_board)
            num_pieces = len(final_board.piece_map())
            if (game.headers["Result"] == "1-0"): # just look at white wins for now, as convention
                end_node = game.end()
                node = end_node
                for _ in range(5): # go back 5 plies
                    node = node.parent
                if node.board().turn == chess.WHITE:
                    node = node.parent
                # display(node.board())
                stockfish.set_fen_position(node.board().fen())
                evaluation = stockfish.get_evaluation()
                # print(stockfish.get_evaluation())
                if ((evaluation['type'] == 'cp') and (evaluation['value']) > 300) or ((evaluation['type'] == "mate") and (evaluation['value'] > 0)): # white is winning
                    endplay_games += 1

print(num_games, endplay_games)

illegal san: 'e1' in r7/3RK1P1/2k5/8/8/8/8/8 b - - 2 80 while parsing <Game at 0x73d306b35c90 ('?' vs. '?', '????.??.??' at '?')>
no matching legal move for e2e4 (e2 -> e4) in 8/2P5/4q3/K7/4k3/8/8/4Q3 b - - 24 86 while parsing <Game at 0x73d307fa2800 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'ba2' in 8/8/8/8/kr6/4K3/p7/3R4 w - - 2 65 while parsing <Game at 0x73d304178880 ('?' vs. '?', '????.??.??' at '?')>
illegal san: 'f1' in 4k3/5p2/2K2P2/4P3/8/8/8/8 b - - 2 78 while parsing <Game at 0x73d3041291e0 ('?' vs. '?', '????.??.??' at '?')>
no matching legal move for e2e4 (e2 -> e4) in 8/3k2P1/5K2/5R2/r7/8/8/8 b - - 0 75 while parsing <Game at 0x73d3041b4490 ('?' vs. '?', '????.??.??' at '?')>
no matching legal move for e2e4 (e2 -> e4) in 5R2/8/8/4r3/8/3K2k1/6p1/8 w - - 8 73 while parsing <Game at 0x73d3075c1d20 ('?' vs. '?', '????.??.??' at '?')>
no matching legal move for e2e4 (e2 -> e4) in 7k/8/4R1KP/8/8/8/8/8 b - - 0 99 while parsing <Game at 0x73d3163af100 ('?' vs. '?', '????.??